# Extract Cabled Receiver Data

### What you'll need before starting:
* A folder of comma-separated .txt files of cabled receiver data metadata to be uploaded
* Specify the database_conn_string.auth file in the ipython-utilities/dbtool folder to connect to the database you want to connect and insert the data into

Run the script below to set up the database connection.

In [ ]:
# Imports. Run this every time.
import dbtools as dbt 
import pprint
from dbtools.connect_db import get_engine, test_engine_connection
from IPython.display import display, HTML
from ipywidgets import *
from plottools.map_drawer import draw_map
from dbtools.load_styles import load_pygment_style
from dbtools.common import display_sql, display_python

load_pygment_style('native')
# Make a DB connection
engine = get_engine() # use defaults in dbtools/database_conn_string.auth
test_engine_connection(engine)

# Validate environmental receiver data

Enter the directory path of the folder of data files and the institution code below. Run the cell below to add the data collected by the receivers whenever a detection was absent to the database.

In [ ]:
import dbtools.extract_cabled_receivers as ecr
import pandas as pd
from os import listdir
from os.path import isfile, join

directorypath = '/Users/jphillips/Desktop/boosbi_data'
headers = ['receiver', 'sequence', 'datetime', 'info', 'detection_count', 'ping_count', 'line_voltage', 'battery_voltage', 'battery_used', 'current_consumption', 'temperature', 'detection_memory_used', 'raw_memory_used', 'tilt_information', 'hex']

df2 = pd.DataFrame(columns=headers)

textfiles = []
for file in os.listdir("/{0}".format(directorypath)):
    if file.endswith(".txt"):
        textfiles.append(os.path.join("/{0}".format(directorypath), file))
        
print "Please wait for operation to finish \n"
for entry in textfiles:
    df2 = df2.append(ecr.check_detection_data_integrity(engine, entry, 'environment')) 
tablename = ecr.create_new_detection_tablename(engine, df2, 'environment')

#Enter in the code of the institution that the data belongs to
institutioncode = 'boosbi'

print "finished"

# Add environmental data to database

Run the following cell to add the environmental data recorded by the receivers to the database specified in the database_conn_string.auth file. It will create a table in the c_environmental_data_yymm_to_yymm_txt format.

In [ ]:
import dbtools.extract_cabled_receivers as ecr

print ecr.insert_environmental_data_into_database(df2, engine, tablename, institutioncode, 10000)

# Validate the detection data

In the cell below, enter in the path to the directory where the receiver data files are located, the institution code that the data belongs to, and then run it to allow the script to process the data.

In [ ]:
import dbtools.extract_cabled_receivers as ecr
import pandas as pd
from os import listdir
from os.path import isfile, join
    
# Insert the paths to your files in the following format as shown in the following example:
# filepaths = ['/Users/name/file1.txt', '/Users/name/file2.txt']
# Note that this tool can only take in .txt files currently
directorypath = '/Users/jphillips/Desktop/boosbi_data'

detection_headers = ['receiver', 'sequence', 'datetime', 'sensor_tag', 'A2D', 'hex']

df = pd.DataFrame(columns=detection_headers)

textfiles = []
for file in os.listdir("/{0}".format(directorypath)):
    if file.endswith(".txt"):
        textfiles.append(os.path.join("/{0}".format(directorypath), file))
        
print "Please wait for operation to finish \n"
for entry in textfiles:
    df = df.append(ecr.check_detection_data_integrity(engine, entry, 'detection'))  
tablename = ecr.create_new_detection_tablename(engine, df2, 'detection')

#Enter in the code of the institution that the data belongs to
institutioncode = 'boosbi'

print "finished"

# Add detections to database

The following cell will take the detections that the script above parsed and add them to the database specified in the database_conn_string.auth file. It will create a table based on the dates of the detections found in a c_detections_yymm_to_yymm_txt format.

In [ ]:
import dbtools.extract_cabled_receivers as ecr

print ecr.insert_detections_into_database(df, engine, tablename, institutioncode, 10000)